TODO: Figure out ravel() 1d array problem

In [10]:
import pandas as pd
import numpy as np
import os
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from __future__ import division
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
labels = pd.read_csv('stocknews//labels.csv', header=None)

http://stackoverflow.com/questions/28382735/python-pandas-does-not-read-the-first-row-of-csv-file

`pd.read_csv` was cutting off the first row of labels

In [12]:
# Confirm size of labels to make sure data loaded correctly
labels.shape

(1989, 1)

In [13]:
str_tokens_all = pd.read_csv('stocknews//tokens_str_all.csv', dtype=str, keep_default_na=False)

In [14]:
str_tokens_all.head(2)

,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,georgia downs two russian warplanes as countri...,breaking musharraf to be impeached,russia today columns of troops roll into south...,russian tanks are moving towards the capital o...,afghan children raped with impunity official s...,150 russian tanks have entered south ossetia w...,breaking georgia invades south ossetia russia ...,the enemy combatent trials are nothing but sha...,georgian troops retreat from osettain capital ...,did the prep georgia for war with russia,...,georgia invades south ossetia if russia gets i...,al qaeda faces islamist backlash,condoleezza rice the us would not act to preve...,this is busy day the european union has approv...,georgia will withdraw 000 soldiers from iraq t...,why the pentagon thinks attacking iran is bad ...,caucasus in crisis georgia invades south ossetia,indian shoe manufactory and again in series of...,visitors suffering from mental illnesses banne...,no help for mexico kidnapping surge
1,why wont america and nato help us if they wont...,bush puts foot down on georgian conflict,jewish georgian minister thanks to israeli tra...,georgian army flees in disarray as russians ad...,olympic opening ceremony fireworks faked,what were the mossad with fraudulent new zeala...,russia angered by israeli military sale to geo...,an american citizen living in ossetia blames a...,welcome to world war iv now in high definition,georgia move mistake of monumental proportions,...,israel and the us behind the georgian aggression,do not believe tv neither russian nor georgian...,riots are still going on in montreal canada be...,china to overtake us as largest manufacturer,war in south ossetia pics,israeli physicians group condemns state torture,russia has just beaten the united states over ...,perhaps the question about the georgia russia ...,russia is so much better at war,so this is what it come to trading sex for food


In [15]:
str_tokens_all.shape

(1989, 25)

In [16]:
def replace_num(element):
    return ' '.join([('numero' if k.isdigit() else k) for k in element.split()])

http://stackoverflow.com/questions/6905636/python-conditional-list-joins

Instead of a digit, use `'NUMERO'` as it is resistant to stemming/lemmatizing.  It's possible that headlines with numbers might contain some information.  This will prevent preprocessing from discarding the information.

### Tag parts-of-speech and lemmatize the text

In [17]:
sample = str_tokens_all.iloc[0,0]

In [18]:
from nltk import pos_tag
from nltk.tokenize import word_tokenize
 
s = "This is a simple sentence"
tokens = word_tokenize(s) # Generate list of tokens
tokens_pos = pos_tag(tokens)
tokens_pos

[('This', 'DT'),
 ('is', 'VBZ'),
 ('a', 'DT'),
 ('simple', 'JJ'),
 ('sentence', 'NN')]

In [19]:
type(tokens)

list

In [20]:
# Need to reconcile treebank and Wordnet tags
# http://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python

In [21]:
from nltk.corpus import wordnet

def get_wordnet_pos(tag):
    if tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('R'):
        return wordnet.ADV
    elif tag.startswith('S'):
        return wordnet.ADJ_SAT
    else:
        return wordnet.NOUN
        # needs something there or else key error

In [22]:
p_o_s = {
    'N' : wordnet.NOUN,
    'V' : wordnet.VERB,
    'J' : wordnet.ADJ,
    'S' : wordnet.ADJ_SAT,
    'R' : wordnet.ADV
}

In [23]:
part = {
    'N' : 'n',
    'V' : 'v',
    'J' : 'a',
    'S' : 's',
    'R' : 'r'
}

In [24]:
def convert_tag(penn_tag):
    '''
    convert_tag accepts the first letter ofa Penn part-of-speech tag
    from nltk.pos_tag() and then uses a dict to converts it to the 
    appropriate WordNet tag.
    '''
    if penn_tag in part.keys():
        return part[penn_tag]
    else:
        return 'n'

In [25]:
tokens_pos[1][1][0] in part.keys()

True

In [27]:
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

In [30]:
# Simple function to tag part of speech (pos) to a sentence and return lemmatized word.
# However, cannnot do it all in one step, need to pos tag before splitting sentence up.

def tag_and_lem(element):
    sent = pos_tag(element.split()) # list of tuples [('token', 'Treebank tag')...]
    return ' '.join([wnl.lemmatize(sent[k][0], get_wordnet_pos(sent[k][1][0]))for k in range(len(sent))])
    #returns str

In [31]:
tag_and_lem(sample)

u'georgia down two russian warplane a country move to brink of war'

# Tag and lemmatize entire corpus

### This will take some time...

In [36]:
# Tag and lem but not merge
lemma_str_tokens_all = str_tokens_all.applymap(tag_and_lem)

In [41]:
lemma_str_tokens_all.head(2)

,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,georgia down two russian warplane a country mo...,break musharraf to be impeach,russia today column of troop roll into south o...,russian tank be move towards the capital of so...,afghan child rap with impunity official say th...,150 russian tank have enter south ossetia whil...,break georgia invades south ossetia russia war...,the enemy combatent trial be nothing but sham ...,georgian troop retreat from osettain capital p...,do the prep georgia for war with russia,...,georgia invades south ossetia if russia get in...,al qaeda face islamist backlash,condoleezza rice the u would not act to preven...,this be busy day the european union have appro...,georgia will withdraw 000 soldier from iraq to...,why the pentagon think attack iran be bad idea...,caucasus in crisis georgia invades south ossetia,indian shoe manufactory and again in series of...,visitor suffer from mental illness ban from ol...,no help for mexico kidnapping surge
1,why wont america and nato help u if they wont ...,bush put foot down on georgian conflict,jewish georgian minister thanks to israeli tra...,georgian army flees in disarray a russian adva...,olympic open ceremony firework fake,what be the mossad with fraudulent new zealand...,russia anger by israeli military sale to georgia,an american citizen living in ossetia blame an...,welcome to world war iv now in high definition,georgia move mistake of monumental proportion,...,israel and the u behind the georgian aggression,do not believe tv neither russian nor georgian...,riot be still go on in montreal canada because...,china to overtake u a large manufacturer,war in south ossetia pic,israeli physician group condemn state torture,russia have just beat the united state over th...,perhaps the question about the georgia russia ...,russia be so much good at war,so this be what it come to trade sex for food


### Merge Cells

In [42]:
lemma_str_tokens_all['merged'] = lemma_str_tokens_all.iloc[:, 0:].apply(lambda x: ' '.join(x.dropna().values.tolist()), axis=1)

In [43]:
lemma_str_tokens_all['merged'] = lemma_str_tokens_all['merged'].apply(replace_num)

## Split data into train and test sets

In [48]:
train_text = lemma_str_tokens_all.merged[0:1493] # train features

In [49]:
test_text = lemma_str_tokens_all.merged[1493:] # test features

In [50]:
train_labels = labels[0:1493].values # train labels

In [51]:
test_labels = labels[1493:].values; # test labels

In [52]:
vectorizer = TfidfVectorizer( max_features=250000, ngram_range=(1, 1), sublinear_tf=True, stop_words='english')

In [53]:
# Only need text, not labels
train_x = vectorizer.fit_transform( train_text )

In [54]:
test_x = vectorizer.transform( test_text )

In [55]:
train_x.shape

(1493, 23750)

In [56]:
test_x.shape

(496, 23750)

### Passive Aggressive Classifier

In [57]:
from sklearn.linear_model import PassiveAggressiveClassifier

In [58]:
classifier = PassiveAggressiveClassifier(n_iter=25)

In [59]:
train_labels.shape

(1493, 1)

In [60]:
train_labels.ravel()

array([0, 1, 0, ..., 1, 1, 1])

In [61]:
classifier.fit(train_x, train_labels.ravel())

PassiveAggressiveClassifier(C=1.0, class_weight=None, fit_intercept=True,
              loss='hinge', n_iter=25, n_jobs=1, random_state=None,
              shuffle=True, verbose=0, warm_start=False)

In [62]:
classifier.score(test_x, test_labels.ravel())

0.49395161290322581

### SGD Classifier

In [63]:
from sklearn.linear_model import SGDClassifier

In [64]:
classifier = SGDClassifier(loss='squared_loss', n_iter=8)

In [65]:
classifier.fit(train_x, train_labels.ravel())

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='squared_loss', n_iter=8, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [66]:
classifier.score(test_x, test_labels.ravel())

0.49596774193548387

### Ridge Classifier

In [67]:
from sklearn.linear_model import RidgeClassifier

In [68]:
clf = RidgeClassifier()

In [69]:
clf.fit(train_x, train_labels.ravel())

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='auto',
        tol=0.001)

In [70]:
clf.score(test_x, test_labels.ravel())

0.49395161290322581

### Gaussian Naive Bayes

In [71]:
from sklearn.naive_bayes import GaussianNB

In [72]:
gnb = GaussianNB()

In [73]:
gnb.fit(train_x.toarray(), train_labels.ravel())

GaussianNB()

In [74]:
gnb.score(test_x.toarray(), test_labels.ravel())

0.53830645161290325

### Support Vector Classifier

In [75]:
from sklearn.svm import SVC

In [76]:
supportvc = SVC()

In [77]:
supportvc.fit(train_x, train_labels.ravel())

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [78]:
supportvc.score(test_x, test_labels.ravel())

0.5161290322580645